In [97]:
import csv
import re
import sys
import os



rows=[]
directory = os.path.join('/home/deepak/StackOverflow Data')
for root,dirs,files in os.walk(directory):
    for file in files:
       if file.endswith(".csv"):
           f=open(directory + "/" + file, 'rt')
           reader = csv.reader(f)
           for row in reader:
                rows.append(row)
       f.close()

len(rows)




247808

In [98]:
# Convert text to lower-case and strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()

    # Replace breaks with spaces
    norm_text = norm_text.replace('\n', ' ')
    norm_text = re.sub("<code>([^>]*)</code>" , " ",norm_text)
    norm_text = re.sub("<*\w+>", " ",norm_text)
    norm_text = re.sub("([^[a-zA-Z])([^\w+\.\w+\.\w+])"," ",norm_text)
    norm_text = re.sub("\s+\w{1}\s+"," ",norm_text)
    norm_text = norm_text.replace('...', ' ')
    return re.split("\s+",norm_text)

# normalize_text(rows[2][2])

cleaned_docs=[normalize_text(row[1]+ row[2]) for row in rows ]

print("Cleaning Done")

Cleaning Done


In [99]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags')

alldocs = []  # will hold all docs in original order

for doc_no, doc in enumerate(cleaned_docs):
    words=doc[1:]
    tags=[doc_no]
    alldocs.append(SentimentDocument(words,tags))
print('%d docs:' % (len(alldocs)))

247808 docs:


In [100]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"


simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
#     Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores),
#     # PV-DBOW 
#     Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores),
#     # PV-DM w/average
        Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores),
]
simple_models[0].build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template

print(simple_models[0])

for model in simple_models[1:]:
    model.reset_from(simple_models[0])
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)



Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


In [86]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
           test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)


In [87]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved


In [101]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes


train_docs = alldocs[0:7500]
test_docs=alldocs[7501:9000]
print("START %s" % datetime.datetime.now())

for epoch in range(passes):
    shuffle(alldocs)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            train_model.train(alldocs)
            duration = '%.1f' % elapsed()
            
#         # evaluate
#         eval_duration = ''
#         with elapsed_timer() as eval_elapsed:
#             err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
#         eval_duration = '%.1f' % eval_elapsed()
#         best_indicator = ' '
#         if err <= best_error[name]:
#             best_error[name] = err
#             best_indicator = '*' 
#         print("%s%f : %i passes : %s %ss %ss" % (best_indicator, err, epoch + 1, name, duration, eval_duration))
#         if ((epoch + 1) % 5) == 0 or epoch == 0:
#             eval_duration = ''
#             with elapsed_timer() as eval_elapsed:
#                 infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs, infer=True)
#             eval_duration = '%.1f' % eval_elapsed()
#             best_indicator = ' '
#             if infer_err < best_error[name + '_inferred']:
#                 best_error[name + '_inferred'] = infer_err
#                 best_indicator = '*'
        print(" %i passes : %s %s" % ( epoch + 1, name + '_inferred', duration))

    print('completed pass %i at alpha %f' % (epoch + 1, alpha))
    alpha -= alpha_delta
    
print("END %s" % str(datetime.datetime.now()))

START 2016-10-02 20:06:20.106386
 1 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 621.8
completed pass 1 at alpha 0.025000
 2 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 575.2
completed pass 2 at alpha 0.023800
 3 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 559.3
completed pass 3 at alpha 0.022600
 4 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 555.7
completed pass 4 at alpha 0.021400
 5 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 546.4
completed pass 5 at alpha 0.020200
 6 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 539.3
completed pass 6 at alpha 0.019000
 7 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 525.9
completed pass 7 at alpha 0.017800
 8 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 522.7
completed pass 8 at alpha 0.016600
 9 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 535.2
completed pass 9 at alpha 0.015400
 10 passes : Doc2Vec(dm/c,d100,n5,w5,mc2,t4)_inferred 500.1
completed pass 10 at alpha 0.014200
 11 passes : Doc2Vec(dm/c

In [102]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))



for doc 244173...
Doc2Vec(dm/c,d100,n5,w5,mc2,t4):
 [(29743, 0.563130795955658), (41351, 0.5335853695869446), (34357, 0.43929237127304077)]


In [111]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(alldocs[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(alldocs[sims[index][0]].words)))

TARGET (178629): «tycho unresolved dependencies i'm working on project lately which goal it is to build an eclipse plugin for palladio my project is built out of several seperate parts and managed to eliminate all errors but the last package still doesn't want to go through i'm using maven "clean works perfectly the error appears when try maven install with tycho and some parts of palladio i've tried to call mvn clean via the console as well as in eclipse the error stays the same suspect there is something missing/wrong in the pom.xml of the last part of the plugin as all the other parts work and maven can clean/install them without an error message i've been working on this for the past week but cant seem to solve it any help is appreciated the following is the pom (i changed user name project name so don't mind that and here is the error i'm getting feature.xml of myproject.featurepatch feature xml of simucom »

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/c,d100,n5,w5,mc2,t4):

MOST